In [7]:
import os
import requests
from requests.auth import HTTPBasicAuth
import pandas as pd
from tkinter import Tk, filedialog


def obter_autenticacao():
    username = os.getenv("ELOQUA_USERNAME")
    password = os.getenv("ELOQUA_PASSWORD")
    return HTTPBasicAuth(username, password)


def pesquisar_nome(nome_pesquisa, base_url, auth, df):
    url = f"{base_url}/API/REST/2.0/assets/landingPages?search=name='{nome_pesquisa}'"

    response = requests.get(url, auth=auth)

    if response.status_code == 200:
        data = response.json()
        for element in data.get("elements", []):
            if "relativePath" in element and element["relativePath"] == nome_pesquisa:
                id_encontrado = element.get("id")
                print(f"Nome: {nome_pesquisa}, ID encontrado: {id_encontrado}")

                url_detalhes = f"{base_url}/API/REST/2.0/assets/landingPage/{id_encontrado}"
                response_detalhes = requests.get(url_detalhes, auth=auth)

                if response_detalhes.status_code == 200:
                    detalhes = response_detalhes.json()

                    detalhes["isHidden"] = "true"

                    # Obtém o valor do "Redirect URL" da planilha df
                    redirect_url = df[df["Relative Path"] == nome_pesquisa]["Redirect URL"].values[0]
                    detalhes["autoRedirectUrl"] = redirect_url

                    detalhes["autoRedirectWaitFor"] = "10"

                    response_put = requests.put(
                        url_detalhes, json=detalhes, auth=auth)

                    if response_put.status_code == 200:
                        print("Atualização bem-sucedida.")
                    else:
                        print(
                            f"Falha na atualização. Código de status: {response_put.status_code}")

    else:
        print(f"Falha na pesquisa. Código de status: {response.status_code}")


def ler_nomes_excel():
    root = Tk()
    root.withdraw()

    file_path = filedialog.askopenfilename()

    df = pd.read_excel(file_path)
    # Extrai os nomes após a barra ("/") e inclui a barra junto com os nomes
    df["Relative Path"] = "/" + df["Relative Path"].str.split("/").str[-1]
    return df, df["Relative Path"].tolist()


if __name__ == "__main__":
    base_url = "https://secure.p03.eloqua.com"
    auth = obter_autenticacao()

    df, nomes_a_pesquisar = ler_nomes_excel()

    for nome_pesquisa in nomes_a_pesquisar:
        pesquisar_nome(nome_pesquisa, base_url, auth, df)

Nome: /Managing-Stat-Reporting-SSC-Webinar, ID encontrado: 672
Atualização bem-sucedida.
Nome: /Managing-Stat-Reporting-SSC-Webinar-View-Video, ID encontrado: 673
Atualização bem-sucedida.
Nome: /synergy-chinese-s, ID encontrado: 1964
Atualização bem-sucedida.
Nome: /synergy-chinese-s-agenda, ID encontrado: 1965
Atualização bem-sucedida.
Nome: /synergy-chinese-s-speakers, ID encontrado: 1966
Atualização bem-sucedida.
Nome: /synergy-chinese-s-thank-you-event-confirmation, ID encontrado: 1967
Atualização bem-sucedida.
Nome: /synergy-chinese-s-resources, ID encontrado: 1969
Atualização bem-sucedida.
Nome: /synergy-chinese-s-contact, ID encontrado: 1970
Atualização bem-sucedida.
Nome: /synergy-chinese-s-thank-you-landing-pages-enquiry, ID encontrado: 1971
Atualização bem-sucedida.
Nome: /synergy-chinese-t, ID encontrado: 1982
Atualização bem-sucedida.
Nome: /synergy-chinese-t-agenda, ID encontrado: 1983
Atualização bem-sucedida.
Nome: /synergy-chinese-t-speakers, ID encontrado: 1985
Atuali